In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [31]:
from machine import freq
from common import task
import time
import random

In [48]:
frequencies = (
    40000000,
    80000000,
    160000000,
    240000000
)

def bench(cb, repeat, *args, **kwargs):
    for f in frequencies:
        freq(f)
        
        alg_time = []
        for i in range(repeat):
            time_start = time.ticks_us()
            cb(*args, **kwargs)
            time_end = time.ticks_us()
            
            alg_time.append(time.ticks_diff(time_end, time_start))
        
        #print(f'{cb.__name__} at {f/1000000} MHz: best/worse/average: {min(alg_time)}/{max(alg_time)}/{sum(alg_time)/len(alg_time)}')
        
        yield {
            'func': cb.__name__,
            'freq': f,
            'min': min(alg_time),
            'max': max(alg_time),
            'avg': sum(alg_time)/len(alg_time)
        }
        

In [51]:
def bubbleSort(arr_in):
    arr = list(arr_in)
    n = len(arr)
 
    # Traverse through all array elements
    for i in range(n-1):
    # range(n) also work but outer loop will repeat one time more than needed.
 
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j] > arr[j + 1] :
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
    return arr
        

        
@task('1: Bubble sort')
def task_1_bubble_sort():
    print(f'bubbleSort([5, 9, 8, 5, 6]) => {bubbleSort([5, 9, 8, 5, 6])}')
    
    element_counts = (5, 10, 50, 100, 200)
    
    for cnt in element_counts:
        test_set = tuple( (random.randint(0, 100000) for i in range(cnt)) )
        
        print(f'\nTest bubble sort for {cnt} elements:')
        for i in bench(bubbleSort, 20, test_set):
            print(f'freq: {i["freq"]}, min/max/avg: {i["min"]}/{i["max"]}/{i["avg"]}')
        #print(tuple())
        
task_1_bubble_sort()    

**************************************************
Start Of Task #1: Bubble sort
**************************************************
bubbleSort([5, 9, 8, 5, 6]) => [5, 5, 6, 8, 9]

Test bubble sort for 5 elements:
freq: 40000000, min/max/avg: 1483/2392/1756.1
freq: 80000000, min/max/avg: 611/1138/650.0999
freq: 160000000, min/max/avg: 412/929/444.6
freq: 240000000, min/max/avg: 342/866/372.65

Test bubble sort for 10 elements:
freq: 40000000, min/max/avg: 3110/4008/3186.25
freq: 80000000, min/max/avg: 1554/2084/1607.2
freq: 160000000, min/max/avg: 886/1398/919.3
freq: 240000000, min/max/avg: 657/1185/689.5

Test bubble sort for 50 elements:
freq: 40000000, min/max/avg: 59726/60626/59839.4
freq: 80000000, min/max/avg: 29815/30341/29870.55
freq: 160000000, min/max/avg: 14999/15517/15044.25
freq: 240000000, min/max/avg: 10065/10592/10102.6

Test bubble sort for 100 elements:
.freq: 40000000, min/max/avg: 227244/228119/227393.0
freq: 80000000, min/max/avg: 113434/113955/113516.6
freq: 16000